In [1]:
import os
import random
from os.path import join
from time import time

import numpy as np
import pandas as pd
import torch
from torch import optim
from torch.nn import BCELoss
from torch.optim import Adam
from torchvision.utils import save_image

from models.daae_final import DAE
from models.dis_z import DIS_Z
from models.activity_recognition import *
from utils.function import *
# from matplotlib import pyplot as plt
# %pylab inline

In [2]:
class Args:
    def __init__(self):
        self.root = '../data'
        self.batchSize = 64
        self.maxEpochs = 200
        self.nz = 400
        self.lr = 1e-4
        self.fSize = 64
        self.outDir = 'data/experiments/DAAE_FINAL'
        self.commit = 'eval'
        self.alpha = 1.0
        self.M = 5
        self.loss = 'MSE' #'BCE'
        self.loadDAE = False
        # self.loadSVM = False    
        self.load_DAE_from = None
        self.evalMode = False
        self.comment = ''
        self.momentum = 0.1
        self.c = 0.01
        # self.svmLR = 1e-4
        self.Ntest = 100
        self.gpuNo = 0
        self.multimodalZ = False
        self.dataset = 'OPPO' # 'PAMAP2' or 'OPPO'
        self.corr = 'Both' # options: Gaussian, ZeroMask, ConsecutiveZeros, Both
        self.sigma = [0.1, 3, 8] # Example: Gaussian: [0.1], ConsecutiveZeros:[3, 10], Both: [0.1, 3, 10]
        self.random_seed = 0
        self.train_split = 0.8

In [3]:
def build_dis(args, dae, multimodalZ):
    if not multimodalZ:
        print('\n ** USING NORMAL PRIOR **')
        prior = dae.norm_prior
        NZ = args.nz
    else:
        print('\n ** USING MULTIMODAL PRIOR **')
        prior = dae.multi_prior
        NZ = 2
    dis = DIS_Z(nz=NZ, prior=prior)

    return dis, NZ

In [4]:
act_labels_txt = ['std', 'wlk', 'sit', 'lie', 'null']

In [8]:
def train_dae(args, exDir, trainLoader, testLoader):
    dae = DAE(nz=args.nz, corr=args.corr, sigma=args.sigma, dataset=args.dataset, fSize=args.fSize, multimodalZ=args.multimodalZ) #sigma=level of corruption
    dis, NZ = build_dis(args, dae=dae, multimodalZ=args.multimodalZ)
    
    if dae.useCUDA:
        torch.cuda.set_device(args.gpuNo)
        dae.cuda()
        dis.cuda()
    
    save_input_args(exDir, args)  #save training opts
    save_exp_details(args, exDir)

    # #Create optimizers
    optimDAE = optim.RMSprop(dae.parameters(), lr = args.lr, momentum=args.momentum)
    optimDIS = optim.RMSprop(dis.parameters(), lr = args.lr, momentum=args.momentum)

    # #Keeping track of training
    losses = {'enc': [], 'rec': [], 'dis':[], 'test rec':[]}

    # xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)

    with torch.autograd.set_detect_anomaly(True):
        for e in range(args.maxEpochs):

            epochEncLoss=0
            epochRecLoss=0
            epochDisLoss=0

            for i, data in enumerate(trainLoader):
                # print(i)

                T = time()

                dae.train()
                dis.train()

                x, y = prep_data(data, useCUDA=dae.useCUDA)
                # print(x.shape)
            
                # get outputs
                zFake, xRec = dae.forward(x)

                # clac losses
                recLoss = dae.rec_loss(xRec, x, loss=args.loss)  #loss='BCE' or 'MSE'
            
                optimDAE.zero_grad()
                recLoss.backward()
                optimDAE.step()            
            
                zFake, xRec = dae.forward(x)    
            
                disLoss = dis.dis_loss(zFake)
                #do updates

                optimDIS.zero_grad()
                disLoss.backward()
                optimDIS.step()
            
                encLoss = dis.gen_loss(zFake)
            
                optimDAE.zero_grad()
                encLoss.backward()
                optimDAE.step()                   


                epochEncLoss+=encLoss.detach().cpu().numpy()
                epochRecLoss+=recLoss.detach().cpu().numpy()
                epochDisLoss+=disLoss.detach().cpu().numpy()
            
                if i%100 == 0:
                    print('[%d, %d] enc: %0.5f, rec: %0.5f, dis: %0.5f, time: %0.3f' % (e, i, encLoss.detach().cpu().numpy(), recLoss.detach().cpu().numpy(), disLoss.detach().cpu().numpy(), time() - T))
                    

            # storing losses for plotting later
            losses['enc'].append(epochEncLoss/i)
            losses['rec'].append(epochRecLoss/i)
            losses['dis'].append(epochDisLoss/i)

            #### Test
            dae.eval()
            dis.eval()

#             #get test outuputs and losses
#             xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)
#             zTest, recTest = dae.forward(xTest)  #N.B. corruption in here
#             recLossTest = dae.rec_loss(recTest, xTest)
#             #Plot losses
#             # losses['test rec'].append(recLossTest.data[0])
#             losses['test rec'].append(recLossTest.detach().cpu().numpy())

#             if e > 0: #only one point for test rec otherwise
#                 plot_losses(losses, exDir, epochs=e+1)
#                 plot_norm_losses(losses, exDir)

            #save parameters
            dae.save_params(exDir)
            dis.save_params(exDir)

            # #Save images of original and rec
            # save_image(xTest.data, join(exDir, 'original.png'))
            # save_image(recTest.data, join(exDir, 'rec.png'))

            # #Save samples
            # sampleDir = join(exDir,'epoch_'+str(e))
            # os.mkdir(sampleDir)
            # print('sample dir:', sampleDir)
            # dae.sample_x(args.M, sampleDir)

In [9]:
# if __name__ == "__main__":
args = Args()

random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

if args.dataset == 'PAMAP2':
    X_train, X_test, y_train, y_test = prepare_data_PAMAP2(args) # x.shape = (94895, 1, 27, 171)
elif args.dataset == 'OPPO':
    X_train, X_test, y_train, y_test = prepare_data_OPPO()
    
trainLoader, testLoader = prepare_dataloaders(args, X_train, X_test, y_train, y_test)
    
exDir = make_new_folder(args.outDir)
print('Outputs will be saved to:', exDir)

 ..from file ../../../../../data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
Outputs will be saved to: data/experiments/DAAE_FINAL/Ex_8


In [ ]:
train_dae(args, exDir, trainLoader, testLoader)